

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/edit/master/tutorials/streamlit_notebooks/healthcare/NER_PROFESSIONS_ES.ipynb)




# **Detect Professions and Occupations in Spanish text**

## 1. Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## 2. Select the DL model

In [4]:
# If you change the model, re-run all the cells below.
# Applicable models: meddroprof_scielowiki
MODEL_NAME = "meddroprof_scielowiki"

## 3. Some sample examples

In [5]:
# Enter examples to be transformed as strings in this list
text_list = [
    """Paciente varón de 42 años que acude al servicio de Urgencias de su hospital acompañado de las Fuerzas del Orden Público por presentar 
    una actitud hostil y desconfiada hacia su padre, permaneciendo aislado en su habitación desde hace un mes. 
    Se decide ingreso hospitalario en Salud Mental. 
    ANTECEDENTES Antecedentes personales y familiares: niega antecedentes personales y familiares de interés. 
    No reacciones alérgicas medicamentosas. 
    Nunca ha estado en tratamiento psiquiátrico, aunque, al parecer, según los datos aportados por familiares, viene presentando síntomas psicóticos desde hace al menos cinco años. 
    Se trata de un varón divorciado, con una hija de 13 años con la que refiere contacto esporádico. 
    Actualmente en desempleo . 
    Sus familiares verbalizan que desde hace unos cinco años –coincidiendo con la ruptura matrimonial, la pérdida de empleo Fuerzas del Orden Público."""]

## 4. Define Spark NLP pipeline

In [9]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The model was trained with the bert_portuguese_base_cased embeddings,
# we need to it.
embeddings = WordEmbeddingsModel.pretrained("embeddings_scielowiki_300d", "es", "clinical/models")\
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained(MODEL_NAME, "es", "clinical/models") \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol('ner')

ner_converter = NerConverterInternal() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[document_assembler, 
                                tokenizer,
                                embeddings,
                                ner_model,
                                ner_converter])

embeddings_scielowiki_300d download started this may take some time.
Approximate size to download 351.2 MB
[OK!]
meddroprof_scielowiki download started this may take some time.
[OK!]


## 5. Run the pipeline

In [10]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(text_list,StringType()).toDF('text')
result = nlp_pipeline.fit(df).transform(df)

result.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|                                                                                               token|                                                                                        

## 6. Visualize results

In [11]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)